# 🎨 Day 2 실습 3: 프롬프트 & 쿼리 엔지니어링 기초

## 🎯 학습 목표
- **프롬프트 엔지니어링 핵심 기법** 실습
- **기본 쿼리 개선 방법** 학습

## 💡 왜 프롬프트 & 쿼리 엔지니어링인가?

### RAG 시스템의 핵심 = LLM 활용
```
Query Refinement → LLM으로 질문 개선
Multi-Query → LLM으로 질문 분해
Answer Generation → LLM으로 답변 생성
```

**좋은 프롬프트 = 좋은 쿼리 = 좋은 RAG!**

## 📋 실습 구성 (45분)

1. **Part 1**: 프롬프트 기초 (10분) - 실습 1
2. **Part 2**: 고급 기법 (20분) - 실습 2, 3
3. **Part 3**: 기본 쿼리 개선 (15분) - 실습 4, 5

In [ ]:
# 라이브러리 설치
!pip install -q transformers torch accelerate sentencepiece

print("✅ 라이브러리 설치 완료!")

In [ ]:
# EXAONE 2.4B 모델 로딩
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

print("🤖 LLM 모델 로딩 중...")

model_name = 'LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map='auto'
)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"✅ 모델 로딩 완료! (Device: {device})")
print(f"   모델: {model_name}")

# Helper function
def generate_text(prompt, max_tokens=200, temperature=0.7, top_p=0.9):
    """LLM으로 텍스트 생성"""
    inputs = tokenizer(prompt, return_tensors='pt').to(model.device)
    input_length = inputs.input_ids.shape[1]
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            temperature=temperature,
            top_p=top_p,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated_tokens = outputs[0][input_length:]
    response = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()
    return response

print("\n✅ generate_text() 함수 준비 완료!")

## 📚 Part 1: 프롬프트 기초 (10분)

### Zero-shot vs Few-shot

**Zero-shot**: 예시 없이 바로 질문
```
질문: "파스텔이 뭐야?"
```

**Few-shot**: 예시를 먼저 보여주고 질문
```
예시 1: 입력 "애플" → 출력 "기술 회사"
예시 2: 입력 "아마존" → 출력 "전자상거래 회사"
질문: "파스텔" → ?
```

### Temperature 파라미터

- **낮음 (0.1~0.3)**: 정확하고 일관된 답변 (검색, 분류)
- **중간 (0.7)**: 균형잡힌 답변 (일반적인 대화)
- **높음 (1.0~1.5)**: 창의적이고 다양한 답변 (글쓰기, 브레인스토밍)

### System/User/Assistant 메시지

- **System**: LLM의 역할/성격 정의
- **User**: 사용자 질문
- **Assistant**: LLM 응답

In [ ]:
# Zero-shot vs Few-shot 비교
query = "파스텔"

# Zero-shot
zero_shot_prompt = f"'{query}'이 무엇인지 한 문장으로 설명하세요."

# Few-shot
few_shot_prompt = """다음 예시를 참고하여 답변하세요.

예시 1:
입력: "애플"
출력: "전자제품을 만드는 기술 회사입니다."

예시 2:
입력: "아마존"
출력: "온라인 쇼핑 서비스를 제공하는 전자상거래 회사입니다."

이제 다음 입력에 답하세요:
입력: "파스텔"
출력:"""

print("🔍 Zero-shot vs Few-shot 비교\n")
print("=" * 60)

print("\n1️⃣ Zero-shot 결과:")
zero_result = generate_text(zero_shot_prompt, max_tokens=50, temperature=0.7)
print(f"   {zero_result}")

print("\n2️⃣ Few-shot 결과:")
few_result = generate_text(few_shot_prompt, max_tokens=50, temperature=0.7)
print(f"   {few_result}")

print("\n" + "=" * 60)
print("\n💡 관찰:")
print("   - Few-shot은 예시 형식을 따라 더 구조화된 답변")
print("   - Zero-shot은 자유로운 형식")

## 🎯 실습 1: Temperature 실험

### 학습 목표
- Temperature 값에 따른 응답 차이 체험
- RAG에서 적절한 Temperature 선택 능력

### 실습 내용
같은 질문에 대해 **3가지 Temperature**로 실험하고 결과를 비교하세요!

In [ ]:
# 여기를 수정하세요! 👇
# TODO: 다양한 temperature 값을 실험해보세요

my_prompt = """다음 복합 질문을 3개의 하위 질문으로 분해하세요.

복합 질문: "주말에 파스텔 공연 보고 호텔 예약도 변경하고 싶어"

하위 질문:
1."""

temperatures = [0.1, 0.7, 1.5]  # TODO: 원하는 값으로 수정

print("🧪 Temperature 실험\n")
print("=" * 70)
print(f"\n프롬프트: {my_prompt[:80]}...\n")

results = {}

for temp in temperatures:
    print(f"\n🌡️ Temperature = {temp}")
    print("-" * 70)
    
    result = generate_text(my_prompt, max_tokens=150, temperature=temp)
    results[temp] = result
    
    print(f"결과:\n{result}")
    print()

print("=" * 70)
print("\n💡 분석해보세요:")
print(f"   - Temperature {temperatures[0]} (낮음): 일관되고 정확한가요?")
print(f"   - Temperature {temperatures[1]} (중간): 균형있는 답변인가요?")
print(f"   - Temperature {temperatures[2]} (높음): 창의적이지만 산만한가요?")
print("\n🎯 RAG Sub-query 생성에는 어떤 값이 좋을까요?")
print("   → 정확성이 중요 → 낮은 Temperature (0.1~0.3) 추천!")

## 🚀 Part 2: 고급 프롬프트 기법 (20분)

### Chain-of-Thought (CoT)

**개념**: LLM에게 "단계별로 생각하도록" 지시

**일반 프롬프트**:
```
질문: 8 + 7 × 3 = ?
```

**CoT 프롬프트**:
```
질문: 8 + 7 × 3 = ?
단계별로 생각해봅시다:
1단계: 먼저 곱셈을 계산
2단계: 그 다음 덧셈을 계산
```

### Few-shot CoT

예시에 추론 과정까지 포함:
```
예시:
질문: 5 + 3 × 2 = ?
추론:
1. 곱셈 먼저: 3 × 2 = 6
2. 덧셈: 5 + 6 = 11
답: 11

이제 풀어보세요:
질문: 8 + 7 × 3 = ?
추론:
```

### Role Prompting

LLM에게 특정 역할 부여:
```
"당신은 검색 전문가입니다."
"당신은 한국어 교사입니다."
"당신은 RAG 시스템 설계자입니다."
```

→ 역할에 맞는 답변 스타일과 전문성

In [ ]:
# CoT 기법 데모
question = """다음 복합 질문에서 필요한 정보가 몇 개인가요?

"주말에 파스텔 공연 보고, 호텔 체크인도 변경하고, 카페 체험도 하고 싶어"""

# 일반 프롬프트
normal_prompt = question + "\n\n답:"

# CoT 프롬프트
cot_prompt = question + """\n\n단계별로 생각해봅시다:
1단계: 질문을 분석하여 요구사항 나열
2단계: 각 요구사항 세기
3단계: 최종 답 도출

답:"""

print("🔍 일반 vs CoT 비교\n")
print("=" * 70)

print("\n1️⃣ 일반 프롬프트:")
normal_result = generate_text(normal_prompt, max_tokens=50, temperature=0.3)
print(f"   {normal_result}")

print("\n2️⃣ CoT 프롬프트:")
cot_result = generate_text(cot_prompt, max_tokens=150, temperature=0.3)
print(f"   {cot_result}")

print("\n" + "=" * 70)
print("\n💡 CoT의 장점:")
print("   - 추론 과정을 명시적으로 보여줌")
print("   - 복잡한 문제 해결에 효과적")
print("   - 답변의 신뢰도 향상")

## 🎯 실습 2: CoT로 복잡한 문제 풀기

### 학습 목표
- CoT 프롬프트 직접 작성하기
- 복잡한 질문 분해 능력 향상

### 실습 내용
복합 질문을 받아 **CoT 프롬프트를 직접 작성**하고 결과를 확인하세요!

In [ ]:
# 여기를 수정하세요! 👇
# TODO: CoT 프롬프트의 추론 단계를 작성하세요

complex_question = """다음 질문을 3개의 하위 질문으로 분해하세요:
"파스텔 호텔 주말 할인 받고, 근처 카페에서 체험하고, 공연도 보고 싶어"""

# TODO: 아래 추론 단계를 채워보세요
my_cot_prompt = complex_question + """\n\n단계별로 생각해봅시다:
1단계: 여기에 첫 번째 단계 작성 (예: 질문에서 요구사항 찾기)
2단계: 여기에 두 번째 단계 작성 (예: 각 요구사항을 명확한 질문으로 변환)
3단계: 여기에 세 번째 단계 작성 (예: 최종 3개 질문 나열)

하위 질문:"""

print("🎨 나만의 CoT 프롬프트 실험\n")
print("=" * 70)
print(f"\n프롬프트:\n{my_cot_prompt}\n")
print("=" * 70)

result = generate_text(my_cot_prompt, max_tokens=200, temperature=0.3)

print("\n📝 결과:")
print(result)

print("\n" + "=" * 70)
print("\n💡 평가해보세요:")
print("   - 3개 하위 질문이 모두 생성되었나요?")
print("   - 각 질문이 명확하고 구체적인가요?")
print("   - CoT 없이 했을 때와 비교해보세요!")

## 🎯 실습 3: Role Prompting 실험

### 학습 목표
- Role Prompting으로 답변 스타일 제어
- RAG 시스템에 적합한 역할 찾기

### 실습 내용
**2가지 다른 역할**을 부여하고 답변 차이를 비교하세요!

In [ ]:
# 여기를 수정하세요! 👇
# TODO: 2가지 역할을 정의하고 비교하세요

task = "다음 복합 질문을 분해하세요: '파스텔 공연 보고 호텔 예약 변경하기'"

# Role 1: 전문가
role1_prompt = """당신은 RAG 시스템 설계 전문가입니다.
복합 질문을 정확하고 효율적으로 분해하는 능력이 뛰어납니다.

""" + task + "\n\n하위 질문:"

# Role 2: 초보자 (TODO: 역할 수정해보세요)
role2_prompt = """당신은 검색 서비스를 처음 배우는 학생입니다.
질문을 간단하고 쉽게 이해하려고 노력합니다.

""" + task + "\n\n하위 질문:"

# TODO: 새로운 역할을 추가해보세요
# role3_prompt = """당신은 ... 입니다.
# ...
# """ + task

print("🎭 Role Prompting 실험\n")
print("=" * 70)

roles = [
    ("전문가", role1_prompt),
    ("초보자", role2_prompt),
    # ("새 역할", role3_prompt)  # TODO: 추가하려면 주석 해제
]

for role_name, prompt in roles:
    print(f"\n🎭 역할: {role_name}")
    print("-" * 70)
    
    result = generate_text(prompt, max_tokens=150, temperature=0.7)
    print(f"결과:\n{result}")
    print()

print("=" * 70)
print("\n💡 비교해보세요:")
print("   - 어떤 역할이 더 정확한 분해를 했나요?")
print("   - 답변 스타일이 역할에 맞나요?")
print("   - RAG에는 어떤 역할이 적합할까요?")
print("\n🎯 RAG 시스템에는 '전문가' 역할이 적합!")

## 🎯 Part 3: RAG용 프롬프트 (15분)

RAG 시스템에 특화된 프롬프트 패턴을 배웁니다!

### RAG의 핵심 프롬프트 2가지

```
사용자 질문
    ↓
1️⃣ Query Rewrite: "더 나은 검색 질문으로 바꾸기"
    ↓
2️⃣ Sub-query Generation: "복합 질문을 여러 개로 쪼개기"
```

### 실습 로드맵
- **실습 4**: Query Rewrite 프롬프트 개선
- **실습 5**: Sub-query 생성 템플릿 만들기

### 1️⃣ Query Rewrite: 검색하기 좋은 질문으로

**문제**: 사용자 질문은 검색에 최적화되지 않음
```
사용자: "파스텔 어떻게 가?"
→ 모호하고 짧음 → 검색 실패
```

**해결**: LLM으로 질문을 명확하고 구체적으로 변환
```
Rewrite: "파스텔 호텔의 위치와 교통편은?"
→ 명확하고 구체적 → 검색 성공
```

**프롬프트 패턴**:
```python
template = f"""
다음 질문을 검색에 적합하도록 더 명확하고 구체적으로 다시 작성하세요.

원본 질문: {user_query}

개선된 질문:
"""
```

In [ ]:
# Query Rewrite 데모
def query_rewrite_prompt(user_query):
    """검색용 질문 개선 프롬프트"""
    return f"""다음 질문을 검색에 적합하도록 더 명확하고 구체적으로 다시 작성하세요.

원본 질문: {user_query}

개선된 질문:"""

# 테스트 질문들
test_queries = [
    "파스텔 어떻게 가?",
    "주말에 할인 있어?",
    "공연 뭐 있나요"
]

print("🔍 Query Rewrite 데모\n")
print("=" * 70)

for i, query in enumerate(test_queries, 1):
    print(f"\n{i}️⃣ 원본 질문: \"{query}\"")
    print("-" * 70)
    
    prompt = query_rewrite_prompt(query)
    rewritten = generate_text(prompt, max_tokens=100, temperature=0.3)
    
    print(f"개선: {rewritten}")

print("\n" + "=" * 70)
print("\n💡 관찰:")
print("   - 모호한 표현 → 구체적 표현")
print("   - 짧은 질문 → 상세한 질문")
print("   - 검색 성공률 향상!")


## 🎯 실습 4: Query Rewrite 프롬프트 개선

### 학습 목표
- Query Rewrite 프롬프트 직접 작성
- Few-shot + CoT 결합하여 성능 향상

### 실습 내용
**예시를 추가**하거나 **CoT를 적용**하여 프롬프트를 개선하세요!

In [ ]:
# 여기를 수정하세요! 👇
# TODO: 프롬프트를 개선하세요 (Few-shot 예시 추가 또는 CoT 적용)

my_test_query = "파스텔 가격"  # TODO: 테스트할 질문 수정

# 기본 프롬프트
basic_prompt = f"""다음 질문을 검색에 적합하도록 개선하세요.

질문: {my_test_query}

개선:"""

# 개선된 프롬프트 (TODO: 아래를 수정하세요!)
improved_prompt = f"""다음 예시를 참고하여 질문을 검색에 적합하도록 개선하세요.

예시 1:
원본: "파스텔 어떻게 가?"
개선: "파스텔 호텔의 위치와 교통편 정보"

예시 2:
원본: "주말 할인"
개선: "파스텔 호텔의 주말 숙박 할인 정책과 조건"

이제 다음 질문을 개선하세요:
원본: {my_test_query}
개선:"""

# TODO: CoT 버전도 만들어보세요!
# cot_prompt = f"""...
# 단계별로 생각해봅시다:
# 1단계: ...
# """

print("🔬 Query Rewrite 프롬프트 비교\n")
print("=" * 70)

print("\n1️⃣ 기본 프롬프트:")
basic_result = generate_text(basic_prompt, max_tokens=80, temperature=0.3)
print(f"   결과: {basic_result}")

print("\n2️⃣ 개선된 프롬프트:")
improved_result = generate_text(improved_prompt, max_tokens=80, temperature=0.3)
print(f"   결과: {improved_result}")

print("\n" + "=" * 70)
print("\n💡 평가:")
print("   - 어떤 프롬프트가 더 나은 결과를 냈나요?")
print("   - Few-shot이 도움이 되었나요?")
print("   - 다른 개선 방법은?


### 2️⃣ Sub-query Generation: 복합 질문 쪼개기

**문제**: 한 질문에 여러 정보 요구
```
"파스텔 공연 보고 호텔도 예약하고 싶어"
→ 공연 정보 + 호텔 예약 정보 필요
```

**해결**: 복합 질문을 여러 개의 단순 질문으로 분해
```
하위 질문 1: 파스텔에서 진행 중인 공연 일정은?
하위 질문 2: 파스텔 호텔 예약 방법은?
→ 각각 검색 → 결과 합침
```

**프롬프트 패턴**:
```python
template = f"""
다음 복합 질문을 3개의 명확한 하위 질문으로 분해하세요.

복합 질문: {user_query}

하위 질문:
1.
2.
3.
"""
```

**💡 이 템플릿은 04번 Multi-Query에서 그대로 사용!**

In [ ]:
# Sub-query Generation 데모
def subquery_generation_prompt(complex_query, num_queries=3):
    """복합 질문을 여러 개로 분해하는 프롬프트"""
    
    
    return f"""다음 복합 질문을 {num_queries}개의 명확한 하위 질문으로 분해하세요.
각 하위 질문은 독립적으로 검색 가능해야 합니다.

복합 질문: {complex_query}

하위 질문:
"""

# 테스트
complex_queries = [
    "파스텔 공연 보고 호텔도 예약하고 싶어",
    "주말에 할인 받고 근처 맛집도 알고 싶어요"
]

print("✂️ Sub-query Generation 데모\n")
print("=" * 70)

for i, query in enumerate(complex_queries, 1):
    print(f"\n{i}️⃣ 복합 질문: \"{query}\"")
    print("-" * 70)
    
    prompt = subquery_generation_prompt(query)
    print(prompt)
    result = generate_text(prompt, max_tokens=200, temperature=0.5)
    
    print(f"하위 질문:\n{result}")

print("\n" + "=" * 70)
print("\n💡 Multi-Query RAG:")
print("   - 각 하위 질문으로 독립 검색")
print("   - 검색 결과를 모두 합침")
print("   - 더 포괄적인 답변 생성!")


## 🎯 실습 5: Sub-query 생성 템플릿 만들기

### 학습 목표
- Sub-query 생성 프롬프트 완성
- **04번 Multi-Query에서 사용할 템플릿** 준비

### 실습 내용
Few-shot 예시와 CoT를 결합하여 **최고의 Sub-query 템플릿**을 만드세요!

**이 템플릿은 04번에서 그대로 사용됩니다!**

In [ ]:
# 여기를 수정하세요! 👇
# TODO: 최고의 Sub-query 생성 템플릿을 만드세요!

my_complex_query = "파스텔 주말 할인 받고 공연 보고 싶어"  # TODO: 수정 가능

# 기본 템플릿
basic_template = f"""다음 질문을 3개로 분해하세요.

질문: {my_complex_query}

하위 질문:
"""

# 최종 템플릿 (TODO: Few-shot + CoT로 개선하세요!)
final_template = f"""당신은 RAG 검색 시스템의 질문 분해 전문가입니다.
다음 예시를 참고하여 복합 질문을 하위 질문으로 분해하세요.

예시:
복합 질문: "파스텔 예약하고 카페 체험도 하고 싶어"
단계별 분석:
1단계: 요구사항 파악 → 예약, 카페 체험
2단계: 각각을 독립 질문으로 변환
하위 질문:
1. 파스텔 호텔 예약 방법과 조건은?
2. 파스텔 또는 근처에서 카페 체험 가능한 곳은?

이제 다음 복합 질문을 분해하세요:
복합 질문: {my_complex_query}
단계별 분석:
1단계: 요구사항 파악 → 
2단계: 독립 질문으로 변환
하위 질문:
"""

print("🏆 Sub-query 템플릿 최종 완성\n")
print("=" * 70)

print("\n1️⃣ 기본 템플릿 결과:")
basic_result = generate_text(basic_template, max_tokens=120, temperature=0.3)
print(basic_result)

print("\n" + "=" * 70)
print("\n2️⃣ 최종 템플릿 결과 (Few-shot + CoT):")
final_result = generate_text(final_template, max_tokens=200, temperature=0.3)
print(final_result)

print("\n" + "=" * 70)
print("\n🎯 이 템플릿을 기억하세요!")
print("   → 04번 Multi-Query에서 그대로 사용!")
print("\n💡 평가:")
print("   - 하위 질문이 명확하고 독립적인가요?")
print("   - 각 질문이 검색 가능한가요?")
print("   - Few-shot과 CoT가 도움이 되었나요?")


## 🎉 03번 완료!

### 배운 내용 정리

**Part 1: 기초**
- Zero-shot vs Few-shot
- Temperature 조절
- System/User/Assistant 메시지

**Part 2: 고급 기법**
- Chain-of-Thought (CoT)
- Role Prompting
- Few-shot CoT 결합

**Part 3: RAG 프롬프트**
- Query Rewrite (질문 개선)
- Sub-query Generation (질문 분해)

### 핵심 패턴

**1. Few-shot이 효과적**
```python
예시를 보여주면 → LLM이 패턴을 따라함
```

**2. CoT로 복잡한 문제 해결**
```python
단계별 추론 → 더 정확한 결과
```

**3. Temperature로 창의성 조절**
```python
낮음(0.1~0.3) → 정확성
높음(0.7~1.5) → 창의성
```

---

### 📚 참고 자료
- [Prompt Engineering Guide](https://github.com/dair-ai/Prompt-Engineering-Guide)
- [OpenAI Prompt Engineering](https://platform.openai.com/docs/guides/prompt-engineering)
- [LangChain Prompts](https://python.langchain.com/docs/modules/model_io/prompts/)

---

**🎯 프롬프트 엔지니어링 기초 완료!**